In [15]:
!haskey(Pkg.installed(), "GaussianDistributions") && Pkg.clone("https://github.com/mschauer/GaussianDistributions.jl")
!haskey(Pkg.installed(), "Kalman") && Pkg.clone("https://github.com/mschauer/Kalman.jl")

false

In [3]:
using Kalman
using StaticArrays

In [4]:
using ForwardDiff
using ForwardDiff: derivative, Dual

In [5]:

M(x) = let x0 = 1.0,
    P0 = 1.0,

    Phi = x,
    b = 0.0,
    Q = 1.0,

    y = NaN,
    H = 1.0,
    R = 1.0
    LinearHomogSystem(x0, P0, Phi, b, Q, y, H, R)
end

θ0 = 1.0

M0 = M(θ0)




Kalman.LinearHomogSystem{Float64,Float64,Float64,Float64,Float64,Float64}(1.0, 1.0, 1.0, 0.0, 1.0, NaN, 1.0, 1.0)

In [6]:
n = 100
k = 10
Y, X0 = sample(n, k, M0)

([2.01703 1.1488 … 2.87387 -1.87169; -2.3329 4.0262 … 2.96398 -0.887573; … ; -4.32943 7.10397 … 8.64568 3.23839; -3.0048 7.82541 … 8.98182 0.859812], [1.80896 3.21035 … 1.08989 0.824194; -1.1101 2.11527 … 2.32186 -1.27393; … ; -4.70355 6.26218 … 8.2848 1.65268; -5.0037 7.14273 … 9.15559 1.1004])

In [7]:
f(t, x, P, Ppred, ll, K) = ll
import Kalman: mappedreturntype
Kalman.mappedreturntype(_, ::Type{typeof(f)}) = Float64

In [8]:
target(x) = sum(sum(MappedKalmanFilter(zip(Base.Iterators.countfrom(1),view(Y, :, j)), M(x), f)) for j in 1:k)
target(v::SVector{1}) = target(v[1])

target (generic function with 2 methods)

In [14]:
θ = 0.7
println(θ)
target2(v) = target(v[1])
for it in 1:10
    θold = θ
    θ = θ - inv(ForwardDiff.hessian(target2, SVector{1}(θ))[1])*ForwardDiff.derivative(target, θ)
    θold == θ && break    
    println(θ)
end
println("Maximum llikelihood estimate: θ = $θ")

0.7
1.119214625197172
0.9794681320562197
0.9878044690148431
0.9877983862786246
0.9877983862765932
Maximum llikelihood estimate: θ = 0.9877983862765932
